# Read Netcdf data

In [1]:
import xarray as xr
import torch
import matplotlib.pyplot as plt

In [2]:
data_path = "/p/project1/training2533/patnala1/WeGenDiffusion/data/2011_t2m_era5_2deg.nc"

In [3]:
ds = xr.open_dataset(data_path)

In [4]:
ds

<xarray.Dataset> Size: 95MB
Dimensions:     (valid_time: 1460, lon: 180, lat: 90)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 12kB 2011-01-01 ... 2011-12-31T18...
  * lon         (lon) float64 1kB 0.0 2.0 4.0 6.0 ... 352.0 354.0 356.0 358.0
  * lat         (lat) float64 720B -89.0 -87.0 -85.0 -83.0 ... 85.0 87.0 89.0
Data variables:
    t2m         (valid_time, lat, lon) float32 95MB ...
Attributes:
    CDI:                     Climate Data Interface version 2.4.4 (https://mp...
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    history:                 Wed Jun 11 23:24:35 2025: cdo remapbil,r180x90 2...
    CDO:                     Climate Data Operators version 2.4.4 (https://mp...

In [5]:
ds['t2m'].isel(valid_time=4).plot()

In [6]:
plt.savefig("sample_fig.png")
plt.clf()

# Forward process
Iteratively adding Gaussian noise   

One step method to emulate multiple timesteps at once

In [7]:
from diffusion import create_diffusion

In [8]:
diffusion = create_diffusion(timestep_respacing="")

In [9]:
sample_map = torch.from_numpy(ds['t2m'].isel(valid_time=4).values)

In [11]:
samples = []
for i in range(0, 1000, 50):
    timestep = torch.tensor(i, dtype=torch.long).to(sample_map.device)
    samples.append(xr.Dataset({
    "t2m": (("lon", "lat"), diffusion.q_sample(sample_map, timestep))
}))

In [15]:
combined = xr.concat(samples,dim="samples")

In [31]:
combined["t2m"].plot(
    col="samples",
    col_wrap=5,
    cmap="viridis",
    cbar_kwargs={
        "orientation": "vertical",   # vertical colorbar at the side
        "pad": 0.05,                  # distance from the plots
        "shrink": 0.8                  # shrink length
    }
)
plt.savefig("forward_process.png")

# Reverse process
Remove noise iteratively